In [ ]:
# todo: 
# - add 'clear popup' actions
# - add clear field text action 
# - expand the window to longer format by default (maybe skip chunking?)

# python3 -m venv .venv
# source .venv/bin/activate


In [1]:
import pydantic
import fastapi

print(f"Running FastAPI version: {fastapi.__version__}")
print(f"Running Pydantic version: {pydantic.__version__}")

Running FastAPI version: 0.115.5
Running Pydantic version: 2.9.1


In [ ]:
# to start the playwright session:
# 
# # PC
## "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9223 --user-data-dir="C:\my-chrome-profile"

# mac
# /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9223 --user-data-dir="~/chrome-debug-profile"


###################################

# to start the server which will communicate with the playwright session:
## cd C:/Users/dusan/OneDrive/Desktop/pp/test/crawl/
## .\.venv\Scripts\activate
## cd v14
## python fastAPIServ.py

## cd ~/Desktop/pp/sync-crawl

## git fetch origin
## git merge origin/main
## git add .
## git commit -m "message"
## git push -u origin main

In [2]:

%load_ext autoreload
%autoreload 2

import helper as hlp
import get_2fa as g2

import hlogger as hlog
import os
from log_config import get_logger

logger = get_logger("Jupyter Notebook")  # Use module name for easier identification

2025-04-05 14:58:12,484 - helper - <module> - Line: 1052 - INFO - Running on Windows


In [6]:



client = 'intuit'
wd = os.getcwd()  # Gets the current working directory
client_folder = 'clients/'+client
# --- Configuration ---
site_description = "Quickbooks Online Website"
site_wide_instructions = """
"""
workflow_id = "add_cust_v98" #unique identifier for the workflow
# workflow_instructions = """Add a new non-inventory type product by navigating to sales->products->New->non-inventory type. Fill out the form with mock data and save the form. Make sure to interact with all form elements to demonstrate and catalog the functionality of the site's function for this workflow.
# Non-inventory type product is a products you buy and/or sell but don’t need to (or can’t) track quantities of, for example, nuts and bolts used in an installation.
# """ #instructions for the workflow
workflow_instructions = """To add a new customer in QuickBooks Online, go to the Sales tab, then select Customers, and click New customer. Fill out the customer’s details in the form and hit Save to finish.
""" #instructions for the workflow
# workflow_instructions = "Add a new customer manually. Follow the icons at the top of the Customer entry form to enter different sections of customer info before saving the form. Enter just basic example data." #instructions for the workflow
#workflow_instructions = "Add sleep mask to cart then go to cart and remove it" #instructions for the workflow

client_url = 'https://sandbox.qbo.intuit.com/app/homepage'
hlp.navigate_to_url_from_metadata(client_url)

extra_instructions = ""
logger_struct = hlog.HierarchicalLogger(client_folder = client_folder)
logger_struct.set_workflow(workflow_id)
logger_struct.set_workflow_value("site_wide_instructions", site_wide_instructions)
logger_struct.set_workflow_value("site_description", site_description)
logger_struct.set_workflow_value("workflow_instructions", workflow_instructions)
logger_struct.set_workflow_value("client_url", client_url)

logger.info(f"Starting workflow {workflow_id} for {client}")



2025-04-05 14:59:19,347 - Jupyter Notebook - <module> - Line: 28 - INFO - Starting workflow add_cust_v98 for intuit


In [7]:
import math
import requests

def fold_screenshot_test(starting_width,starting_height,depth_chunks_to_test,run_rerun_path,workflow_instructions,site_wide_instructions,site_description,directory):

    #run_rerun_path = 'clients/qb\\cust3\\0\\0\\0\\0'
    logger.info(f"Requesting extract_metadata for {client} with starting width {starting_width} and starting height {starting_height}")
    # scans through the page and extracts metadata
    metadata_response = requests.get(f"http://127.0.0.1:8000/extract_metadata?steps={10}&wait_per_step_ms={1000}&overlap_percent=30",)
    if metadata_response.status_code != 200:
        logger.error(f"Failed to fetch metadata. Status code: {metadata_response.status_code}. Message: {metadata_response}")
        raise Exception(f"Failed to fetch metadata. Status code: {metadata_response.status_code}. Message: {metadata_response}")
        
    url_metadata = metadata_response.json()
    url_metadata



    # modified_height,tokens = hlp.largest_height_given_width(max_tokens=1600,orig_width=starting_width,suggested_height=starting_height)

    import copy
    new_height = copy.deepcopy(starting_width)

    full_height = url_metadata['dimensions']['fullPage']['height']

    diff = full_height - new_height
    increment = math.ceil(diff/depth_chunks_to_test)


    fold_tests = []

    i = 1
    while i <= depth_chunks_to_test:
        logger.info(f"Resizing browser to width {starting_width} and height {new_height}")
        resize_browser_result = hlp.resize_browser_window(width=starting_width, height=new_height)
        fold_tests.append({'width':starting_width,'height':new_height,'id':i})

        logger.info(f"Requesting screenshot for chunk {i}")
        screenshot_response = requests.get(
            f"http://127.0.0.1:8000/screenshot",## todo
            params={"output_path": run_rerun_path+f"/{directory}", "overlap_percentage": 0,"max_chunks": 1, 
                        "action_id": None,
                        "candidate_id": None,
                        "single_chunk_override_id":i
                    }
        )
        logger.info(f"Completed screenshot for chunk {i}")
        new_height = new_height + increment
        i += 1
    resize_browser_result = hlp.resize_browser_window(width=starting_width, height=starting_height)

    fold_test_paths = []
    #chunk_file_list =  hlp.get_files_with_extension(run_rerun_path+"/chunks", ".png")
    for c in fold_tests:
        logger.info(f"Resizing screenshot {c['id']}")
        tmp_resize_screenshot_result = hlp.resize_and_crop(
            input_path  = f"{run_rerun_path}/{directory}/chunk_{c['id']}.png",
            output_path = f"{run_rerun_path}/{directory}/resized_chunk_{c['id']}.png"
        )
        c['path'] = run_rerun_path+f"/{directory}/resized_chunk_{c['id']}.png"
        c['file_name'] = f"resized_chunk_{c['id']}.png"
        fold_test_paths.append(c['path'])
        tmp_resize_screenshot_result['id'] = c['id']
        c['new_dimensions'] = {'height':tmp_resize_screenshot_result['new_height'],'width':tmp_resize_screenshot_result['new_width']}
        #print(c['id'],":",'width:',c['new_dimensions'])

###################


    return fold_tests,fold_test_paths,screenshot_response,url_metadata,resize_browser_result

def perform_fold_test_llm(fold_tests,fold_test_paths,screenshot_response,resize_browser_result):

    page_action_json_ranked_fold_test = None

    if len(fold_tests) > 1:
        logger.info(f"Requesting fold test llm process because there are {len(fold_tests)} fold tests")

        page_action_json_ranked_fold_test,action_prompt,raw = hlp.perform_llm_fold_test(fold_tests,fold_test_paths,workflow_instructions,site_wide_instructions,site_description)

        best_image={}
        best_image = max(
            page_action_json_ranked_fold_test,
            key=lambda x: (x['relevant_elements'], x['clarity']),
        )
        for f in fold_tests:
            if f['id'] == best_image['id']:
                best_image['path'] = f['path']
                best_image['file_name'] = f['file_name']
                best_image['resized_dimensions'] = f['new_dimensions']
                best_image['original_dimensions'] = {'height':f['height'],'width':f['width']}
        
        logger.info(f"Resizing window for best image is {best_image['file_name']} with clarity {best_image['clarity']} and {best_image['relevant_elements']} relevant elements")
        
        resize_browser_result = hlp.resize_browser_window(width=best_image['original_dimensions']['width'], height=best_image['original_dimensions']['height'])

    else:
        best_image={}
        best_image['id'] = fold_tests[0]['id']
        best_image['path'] = fold_tests[0]['path']
        best_image['file_name'] = fold_tests[0]['file_name']
        best_image['resized_dimensions'] = fold_tests[0]['new_dimensions']
        best_image['original_dimensions'] = {'height':fold_tests[0]['height'],'width':fold_tests[0]['width']}

    #best_image

    # resize browse to the best fold

    screenshot_data = screenshot_response.json()

    return page_action_json_ranked_fold_test,best_image,screenshot_data,resize_browser_result



In [8]:


import time
import json
import copy
import requests

max_run = 11
max_rerun = 4
max_run_rerun = 4

max_chunks = 10
overlap_percentage = 0
draw_dots = True


run_id = 0
rerun_ct = 0
run_rerun_ct = 0
advice = ''
summary = ''
goal_achieved = False
any_actions_succeeded = False
resize_data = None  # Store resize data between runs

# Main loop

actions_succeeded = False
page_changed = False
overall_goal_success = False

stability_timeout_ms = 10000
stability_window_ms = 1000
check_stability_x_before_error = 5

wait_after_scroll = 500


# 1) Set context
sample_id = logger_struct.set_sample()
logger_struct.set_rerun(rerun_ct)
logger_struct.set_run(run_id)
logger_struct.set_run_retry(run_rerun_ct)

# todo: 
# test what happens when a rerun is needed
# add "generalized instructions" to the actions
import math
import requests

starting_width = 1256
starting_height=1369
depth_chunks_to_test = 2
run_image_double_check = False

# dusan_tracker['goal']=workflow_instructions

#resize_browser_result = hlp.resize_browser_window(width=browser_start_width, height=browser_start_height)
run_rerun_path = ''

last_rerun_advice = ''

while run_id <= max_run and rerun_ct <= max_rerun and not overall_goal_success and run_rerun_ct <= max_run_rerun:

    
    #print(logger.context)
    # deep copy run_rerun_path
    previous_run_rerun_path = copy.deepcopy(run_rerun_path)
    #print(f"Now previous = {previous_run_rerun_path} which is run {run_rerun_path}")
    run_rerun_path = os.path.join(client_folder, workflow_id,str(sample_id),str(rerun_ct),str(run_id),str(run_rerun_ct))
    logger.info(f"Previous run_rerun_path is {previous_run_rerun_path} and current run_rerun_path is {run_rerun_path}")
    #print(f"And Run is now: {run_rerun_path}")
    logger.info(f"Starting run {run_id} rerun {rerun_ct} run_rerun {run_rerun_ct} in {run_rerun_path}")


    start_time = time.time()

    # Set up folders
    
    if run_id == 0:
        summary = ''



        ############################################


        logger.info(f"It's the first run for the sample. Starting fold screenshot test")
        fold_tests,fold_test_paths,screenshot_response,resize_browser_result,url_metadata = fold_screenshot_test(starting_width=starting_width,starting_height=starting_height,depth_chunks_to_test=depth_chunks_to_test,run_rerun_path=run_rerun_path,workflow_instructions=workflow_instructions,site_wide_instructions=site_wide_instructions,site_description=site_description,directory='fold_test')

        logger.info(f"Completed fold screenshot test. Starting fold test LLM analysis for {len(fold_tests)} fold tests")

        page_action_json_ranked_fold_test,best_image,screenshot_data,resize_browser_result = perform_fold_test_llm(fold_tests,fold_test_paths,screenshot_response,resize_browser_result)

        ############################################



        # First run in a rerun sequence OR stalled - get fresh screenshot and metadata
        # NOTICE: this needs to be done only once per rerun sequence (unless the browser is messed with durnin the run)
        
        # url_metadata, screenshot_data = hlp.run_metadata_gather(
        #     output_path=run_rerun_path+"/chunks",
        #     overlap_percentage=overlap_percentage,
        #     max_chunks=max_chunks
        # )

        if rerun_ct == 0:
            logger.info(f"First run in the rerun sequence. Deleting advice")
            advice = ''
        else:
            last_rerun_advice = advice # if this is not the first rerun, and is the first run in the rerun, add previous rerun's advice (from the last step/run of the previous rerun) as the last rerun advice to follow

        resize_data = {
            'resize_browser_result': resize_browser_result,
            #'resize_screenshot_result': resize_screenshot_result,
            'best_image':best_image,
            'url_metadata': url_metadata,
            'screenshot_data': screenshot_data
        }

        logger.info(f"Copying screenshot from fold test folder to chunk folder for best image for resized and non rezised for best image id {best_image['id']}")

        hlp.copy_and_rename_file(f"{best_image['path']}", f"{run_rerun_path}/chunks/resized_chunk_{best_image['id']}.png",delete_original=False)
        hlp.copy_and_rename_file(f"{run_rerun_path}/fold_test/chunk_{best_image['id']}.png", f"{run_rerun_path}/chunks/chunk_{best_image['id']}.png",delete_original=False)

    else:
        logger.info(f"Subsequent run in the same rerun sequence. Reusing data from previous run. Advice length is {len(advice)} characters")
        # Subsequent runs - copy screenshot from temp and reuse resize data


        try: # we write to temp so that we can compare once we have the new screenshot

            # temp was captured for eval in the 0th and all other runs

            logger.info(f"Copying screenshot from temp to new run folder: {previous_run_rerun_path}/temp to {run_rerun_path}/chunks")

            if len(previous_run_rerun_path) == 0:
                logger.error(f"Copying screenshot to new run folder: {previous_run_rerun_path} is empty")
            temp_file_list =  hlp.get_files_with_extension(previous_run_rerun_path+"/temp", ".png")
            for tf in temp_file_list:
                hlp.copy_and_rename_file(
                    f'{previous_run_rerun_path}/temp/{tf}',
                    f'{run_rerun_path}/chunks/{tf}',
                    delete_original=False
                )

            # Reuse resize data from first run
            resize_browser_result = resize_data['resize_browser_result']
            #resize_screenshot_result = resize_data['resize_screenshot_result']
            best_image = resize_data['best_image']
            url_metadata = resize_data['url_metadata']
            screenshot_data = resize_data['screenshot_data']
            
            # hlp.add_step_to_run_data(run_data, 'reused_data', 'Using copied data from previous run', {
            #     "url_metadata": url_metadata,
            #     "screenshot_data": screenshot_data,
            #     "resize_screenshot_result": resize_screenshot_result
            # })
            
        except Exception as e:
            logger.error(f"Error copying screenshot to new run folder: {e}")
            continue




    logger.info(f"Starting LLM analysis with advice length {len(advice)} characters")

    page_action_json_ranked,action_prompt = hlp.perform_llm_analysis(
        extra_instructions,
        site_wide_instructions,site_description,
        run_rerun_path,
        workflow_instructions,
        #resize_screenshot_result, # TODO: change to be broader
        best_image,
        run_id,
        start_time
        ,advice
        ,last_rerun_advice
        ,summary
    )
    
    logger.info(f"Completed LLM analysis. {len(page_action_json_ranked['action_tasks'])} actions to perform")



    if page_action_json_ranked is None:
        logger.error("LLM analysis failed to produce valid results")
        continue
    else:
        if len(page_action_json_ranked['action_tasks']) == 0:
            logger.error("LLM analysis failed to produce valid results")
            continue
    #hlp.add_step_to_run_data(run_data, 'llm_analysis', 'LLM analysis for action planning', page_action_json_ranked)

    # print("\nDrawing elements and creating highlighted screenshot...")
    # print(f"Page action JSON: {json.dumps(page_action_json_ranked, indent=2)}")


    logger_struct.set_run_retry_value("summary_of_steps_so_far", page_action_json_ranked['summary_of_steps_so_far'])
    logger_struct.set_run_retry_value("expected_outcome_hopeful", page_action_json_ranked['expected_outcome_hopeful'])
    logger_struct.set_run_retry_value("_advice_assessment", page_action_json_ranked['_advice_assessment'])
    logger_struct.set_run_retry_value("page_description", page_action_json_ranked['page_description'])


    

## left off
    
    # Prepare actions to perform
    actions_to_perform = []
    for task in page_action_json_ranked.get('action_tasks', []):
        logger_struct.set_action(task['action_id'])
        logger_struct.set_action_value("description", task['description'])
        # Fix the string concatenation by ensuring proper order of operations
        # dusan_tracker.append("    -Action tasks: ID:" + str(task['action_id']) + " Descript: " + str(task['description']))

        for c in task['candidates']:
            logger_struct.set_candidate(c['candidate_id'])
            logger_struct.set_candidate_value(key="element_description", value=c['element_description'])
            logger_struct.set_candidate_value(key="_need_to_click_dropdown_arrow",value=c['_need_to_click_dropdown_arrow'])
            logger_struct.set_candidate_value(key="type_text", value=c.get('type_text', ''))
            logger_struct.set_candidate_value(key="keyboard_action", value=c.get('keyboard_action', ''))
            logger_struct.set_candidate_value(key="image_number", value=c.get('image_number', ''))
            logger_struct.set_candidate_value(key="action", value=c['action'])
            logger_struct.set_candidate_value(key="to_act", value=c['to_act'])

        action_candidates = {
            'description': task['description'],
            'action_id': task['action_id'],
            'candidates': [c for c in task['candidates'] if c.get('to_act', False)]
        }

        actions_to_perform.append(action_candidates)


    # Prepare the action request payload
    action_request_payload = {
        'multiple_steps_required': page_action_json_ranked.get('multiple_steps_required', False),
        'visible_elements_from_instructions': page_action_json_ranked.get('visible_elements_from_instructions', ''),
        'summary_of_steps_so_far': page_action_json_ranked.get('summary_of_steps_so_far', ''),
        'action_tasks': actions_to_perform,
        'error': page_action_json_ranked.get('error', None),
        'page_description': page_action_json_ranked.get('page_description', ''),
        'expected_outcome_hopeful': page_action_json_ranked.get('expected_outcome_hopeful', '')
    }


    # Execute actions
    #response = hlp.execute_actions(action_request_payload, 2)  # The 2 is the wait_time parameter

    # Execute actions using the API server (FastAPI).
    logger.info(f"Sending screenshotting request to API server for {len(action_request_payload['action_tasks'])} actions")
    draw_response = hlp.send_action_request(action_request_payload, wait_time=2,
        run_rerun_path=run_rerun_path,draw_no_action=True)
    

    if run_image_double_check:
        logger.info(f"Double checking the image draw dots")
        coordinate_info_to_review = []
        for a in action_request_payload['action_tasks']:
            coordinate_info_to_review.append({'action_id':a['action_id'],'description':a['description'],'element_description':a['candidates'][0]['element_description'],'coordinates':a['candidates'][0]['coordinates_ready_to_act'],'action':a['candidates'][0]['action']})

        coor_check_results,coor_check_prompt,coor_check_raw = hlp.check_coordinates_llm(workflow_instructions,site_wide_instructions,site_description,run_rerun_path,coordinate_info_to_review)
    else:
        logger.info(f"Skipping double checking the image draw dots")
    

    logger.info(f"Sending action request to API server for {len(action_request_payload['action_tasks'])} actions")
    action_response = hlp.send_action_request(action_request_payload, wait_time=2,
        run_rerun_path=run_rerun_path,draw_no_action=False)

    # not shure if this is needed
    hlp.resize_and_crop(
            f'{run_rerun_path}/dots/final_screenshot.png',
            f'{run_rerun_path}/dots/resized_final_screenshot.png'
        )



    actions_succeeded = False
    page_changed = False
    goal_achieved = False
    overall_goal_success = False

    if action_response:
        logger.info(f"Executed run actions for {len(action_response)} actions")


        if type(action_response[0]['element_metadata'])!=dict:
            logger.error("Element metadata is not a dictionary")
        elif len(action_response[0]['element_metadata']) == 0:
            logger.error("No element metadata in the response")
        else: # Store element metadata for each action and candidate in the logger struct and the log file for the run retry
            logger.info(f"Found {len(action_response[0]['element_metadata'])} out of {len(action_response)} element metadata in the response")
            # Add element metadata to logger_struct in sync with existing matching logic
            for at in page_action_json_ranked['action_tasks']:
                for c in at['candidates']:
                    for r in action_response:
                        if r['action_id'] == at['action_id'] and r['candidate_id'] == c['candidate_id']:
                            logger_struct.set_action(r['action_id'])
                            logger_struct.set_candidate(r['candidate_id'])
                            logger_struct.set_candidate_value(key="element_metadata", value=r['element_metadata'])


        highlighting_metadata = {}
        for at in page_action_json_ranked['action_tasks']:
            #print(at)
            for c in at['candidates']:
                #print(c)
                for r in action_response:
                    if r['action_id'] == at['action_id'] and r['candidate_id'] == c['candidate_id']:
                        # dusan_tracker.append("         -Acted on Action tasks: aID:" + str(r['action_id']) + " cID:" + str(r['candidate_id']))

                        


                        if c.get('action','') != 'keyboard_action' and (c.get("keyboard_action", '') is None or c.get("keyboard_action", '').strip()==''):
                            c['element_metadata'] = r['element_metadata']
                            new_box = copy.deepcopy(c['element_metadata']['boundingBox'])
                            #new_box['y'] = new_box['y'] - c['scroll_to']

                            if at['action_id'] in highlighting_metadata:
                                highlighting_metadata[at['action_id']].append(new_box)
                                
                                    
                            else:
                                highlighting_metadata[at['action_id']] = [new_box]
                        else:
                            c['element_metadata'] = {}
        
        logger.info(f"Creating highlighted screenshot with {len(highlighting_metadata)} boxes")

        file_for_cairo = next((s for s in hlp.get_files_with_extension(run_rerun_path+"/chunks", ".png") if s.startswith("chunk")), None)
        hlp.create_highlighted_screenshot_cairo(
            run_rerun_path=run_rerun_path, # Use the current
            bbox_payload = highlighting_metadata
            ,file_for_cairo = file_for_cairo
        )
        # check for stability before proceeding by checking if the page is stable

        # todo: create stability check
        # sleep for 5 seconds
        time.sleep(2)       
        # stability = False
        # for i in range(check_stability_x_before_error):

        #     screenshot_response = requests.get(
        #         f"http://127.0.0.1:8000/check-stability?timeout_ms={stability_timeout_ms}&window_size_ms={stability_window_ms}",
        #     )
        #     if screenshot_response.json()['is_stable']:
        #         stability = True
        #         break
        # if not stability:
        #     print("Error!!!!!!!!!!!!!!!!!!!: Stability not reached")
        #     break

########################################################
   

        logger.info(f"Requesting extract_metadata with starting width {starting_width} and starting height {starting_height}")

        fold_tests_temp,fold_test_paths_temp,screenshot_response_temp,resize_browser_result_temp,url_metadata_temp = fold_screenshot_test(starting_width=starting_width,starting_height=starting_height,depth_chunks_to_test=depth_chunks_to_test,run_rerun_path=run_rerun_path,workflow_instructions=workflow_instructions,site_wide_instructions=site_wide_instructions,site_description=site_description,directory='fold_test_temp')

        logger.info(f"Completed retro/temp fold screenshot test. Starting fold test LLM analysis for {len(fold_tests_temp)} fold tests")

        page_action_json_ranked_fold_test_temp,best_image_temp,screenshot_data_temp,resize_browser_result_temp = perform_fold_test_llm(fold_tests_temp,fold_test_paths_temp,screenshot_response_temp,resize_browser_result_temp)
        
        logger.info(f"Completed retro/temp fold test screenshot LLM analysis and the dimensions of the best image ID {best_image_temp['id']} are {best_image_temp['original_dimensions']}")


        logger.info(f"Copying screenshot from 'fold_test_temp' folder to 'temp' folder for best image for resized and non rezised for best image id {best_image['id']}")

        hlp.copy_and_rename_file(f"{best_image_temp['path']}", f"{run_rerun_path}/temp/resized_chunk_{best_image_temp['id']}.png",delete_original=False)
        hlp.copy_and_rename_file(f"{run_rerun_path}/fold_test_temp/chunk_{best_image_temp['id']}.png", f"{run_rerun_path}/temp/chunk_{best_image_temp['id']}.png",delete_original=False)

############# STOPED HERE XISOFYSFE
        previous_step_detail = hlp.extract_entities_for_evaluation(page_action_json_ranked)

        logger.info(f"Starting evaluation of the run screenshots")

        eval_json = hlp.evaluate_run_screenshots(
            workflow_instructions,
            site_wide_instructions,site_description,
            run_rerun_path,
            #[resize_screenshot_result, resize_screenshot_result_temp],
            [best_image,best_image_temp],
            previous_step_detail,
            summary,
            start_time
        )
        logger.info(f"Completed evaluation of the run screenshots")

        logger_struct.append_to_open_log_run_retry("Evaluation completed")

        actions_succeeded = eval_json['actions_succeeded']
        page_changed = eval_json['page_changed']
        overall_goal_success = eval_json['overall_goal_success']

        logger_struct.set_run_retry_value("actions_succeeded", eval_json['actions_succeeded'])
        logger_struct.set_run_retry_value("page_changed", eval_json['page_changed'])
        logger_struct.set_run_retry_value("overall_goal_success", eval_json['overall_goal_success'])

        logger_struct.set_run_retry_value("summary", eval_json['summary'])
        logger_struct.set_run_retry_value("run_advice", eval_json['run_advice'])
        logger_struct.set_run_retry_value("_expectations_analysis", eval_json['_expectations_analysis'])
        logger_struct.set_run_retry_value("_precision_analysis", eval_json['_precision_analysis'])
        logger_struct.set_run_retry_value("_error_type", eval_json['_error_type'])

        for a in eval_json['action_tasks']:
            logger_struct.set_action(a['action_id'])
            logger_struct.set_action_value(key="_error_type", value=c.get('_error_type', ''))

            for c in a.get('candidates', []):
                logger_struct.set_candidate(c['candidate_id'])
                logger_struct.set_candidate_value(key="success", value=c.get('success', ''))
                logger_struct.set_candidate_value(key="issues", value=c.get('issues', ''))
                logger_struct.set_candidate_value(key="_candidate_advice", value=c.get('_candidate_advice', ''))


        if not overall_goal_success:
            logger.info(f"Goal not achieved. Continuing to next run. Advice lenght is {len(eval_json['run_advice'])} characters and overall summary is {len(eval_json['summary'])} characters")
            # dusan_tracker.append("    -didn't succeed in goal")
            advice = f"ADVICE FROM PREVIOUS STEPS OR RUNS: Consider this advice from previous attempts to guide you what to do next.: {eval_json['run_advice']}\n"
            summary = f"OVERALL PROCESS SUMMARY: Consider this summary from previous attempts to guide you what to do next.: {eval_json['summary']}\n"

    else:
        logger.error("No response from the action API")

    # track_data["runs"].append(run_data)
    # hlp.write_track_data(track_data, folder_structures['workflow_path'])
    


    # Update run_id based on success of actions
    # If any actions succeeded and can't contune, increment run_id
    if not actions_succeeded and page_changed:  # If we cannot continue (major error)

        logger_struct.append_to_open_log_run_retry("Run needs to reset. Incremeenting rerun count.")

        rerun_ct += 1
        run_id = 0
        run_rerun_ct = 0
        logger_struct.set_rerun(rerun_ct)
        logger_struct.set_run(run_id)
        logger_struct.set_run_retry(run_rerun_ct)

        logger.info(f"PI: {sample_id}{rerun_ct}{run_id}{run_rerun_ct} Run needs to reset because actions didn't succeed and page was changed. Incrementing rerun count to {rerun_ct}. Resetting run_id to {run_id} and run_rerun_ct to {run_rerun_ct}. Navigating to initial state url metadata")

        any_actions_succeeded = False

        # dusan_tracker.append("    -Cannot continue, resetting memory, rerun count: " + str(rerun_ct) + "navigation to initial state url metadata")


        hlp.navigate_to_url_from_metadata(client_url)

        # Do NOT increment run_id here!
    elif actions_succeeded and page_changed:  # If we can continue (minor or no changes)
        

        logger_struct.append_to_open_log_run_retry("Action succeeded, page changed.")
        # dusan_tracker.append(f"    -Can continue actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
        if not overall_goal_success:  
            
            
            logger_struct.append_to_open_log_run_retry("Goal not yet achieved")
            # dusan_tracker.append(f"    -Goal not achieved but progress made, goal_achieved: {goal_achieved}, actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
            run_id += 1  # Move to the next run (correct)
            logger_struct.set_run(run_id)
            run_rerun_ct = 0
            logger_struct.set_run_retry(run_rerun_ct)
            
            logger.info(f"PI: {sample_id}{rerun_ct}{run_id}{run_rerun_ct} Progress made. Goal not yet fully achieved. Incrementing run_id to {run_id} and resetting run_rerun_ct to {run_rerun_ct}")
        else:
            
            logger.info(f"PI: {sample_id}{rerun_ct}{run_id}{run_rerun_ct} Goal achieved. Ending the workflow.")
            logger_struct.append_to_open_log_run_retry("Goal achieved")
    else: # If we can continue but the page hasn't changed
        
        logger_struct.append_to_open_log_run_retry("Page hasn't changed, but can continue at current step.")
        #dont change the run_id but increment the run rerun count
        # dusan_tracker.append(f"    -Page hasn't changed, goal_achieved: {goal_achieved}, actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
        run_rerun_ct += 1
        
        logger.info(f"PI: {sample_id}{rerun_ct}{run_id}{run_rerun_ct} Page hasn't changed, but can continue at current step. Incrementing run_rerun_ct to {run_rerun_ct}")

        logger_struct.set_run_retry(run_rerun_ct)
    

    #todo
    # why do I still get Error in main loop: Key 'summary_of_steps_so_far' already exists in this node; cannot overwrite.

2025-04-05 14:59:26,895 - Jupyter Notebook - <module> - Line: 69 - INFO - Previous run_rerun_path is  and current run_rerun_path is clients/intuit\add_cust_v98\0\0\0\0
2025-04-05 14:59:26,896 - Jupyter Notebook - <module> - Line: 71 - INFO - Starting run 0 rerun 0 run_rerun 0 in clients/intuit\add_cust_v98\0\0\0\0
2025-04-05 14:59:26,897 - Jupyter Notebook - <module> - Line: 86 - INFO - It's the first run for the sample. Starting fold screenshot test
2025-04-05 14:59:26,898 - Jupyter Notebook - fold_screenshot_test - Line: 7 - INFO - Requesting extract_metadata for intuit with starting width 1256 and starting height 1369
2025-04-05 14:59:27,975 - Jupyter Notebook - fold_screenshot_test - Line: 34 - INFO - Resizing browser to width 1256 and height 1256
2025-04-05 14:59:28,694 - Jupyter Notebook - fold_screenshot_test - Line: 38 - INFO - Requesting screenshot for chunk 1
2025-04-05 14:59:28,960 - Jupyter Notebook - fold_screenshot_test - Line: 47 - INFO - Completed screenshot for chunk 1

In [14]:
import json
import copy
from pathlib import Path
import datetime
import uuid

# --- Helper Functions ---

def _convert_datetime_strings_to_objects(data_dict):
    """Recursively convert ISO format datetime strings to datetime objects."""
    if isinstance(data_dict, dict):
        if "created_at" in data_dict and isinstance(data_dict["created_at"], str):
            try:
                data_dict["created_at"] = datetime.datetime.fromisoformat(data_dict["created_at"])
            except ValueError:
                # Handle cases where the string might not be a valid ISO format
                # Or decide to keep it as a string if conversion fails
                pass
        for value in data_dict.values():
            _convert_datetime_strings_to_objects(value)
    elif isinstance(data_dict, list):
        for item in data_dict:
            _convert_datetime_strings_to_objects(item)

def _convert_datetime_objects_to_strings(data_dict):
    """Create a copy of the data with datetime objects converted to ISO format strings."""
    if isinstance(data_dict, dict):
        result = {}
        for key, value in data_dict.items():
            if isinstance(value, datetime.datetime):
                result[key] = value.isoformat()
            else:
                result[key] = _convert_datetime_objects_to_strings(value)
        return result
    elif isinstance(data_dict, list):
        return [_convert_datetime_objects_to_strings(item) for item in data_dict]
    else:
        return data_dict

# --- Main Logic ---

def analyze_workflow_logs(json_file_path, target_workflow_id):
    """
    Analyzes the logger JSON for a specific workflow, calculates stats,
    and extracts cleaned successful paths.

    Args:
        json_file_path (str or Path): Path to the logger.json file.
        target_workflow_id (str): The ID of the workflow to analyze.

    Returns:
        dict: A dictionary containing analysis results, including stats
              and cleaned successful rerun data, or an error message.
              Returns None if the file cannot be read or parsed.
    """
    try:
        with open(json_file_path, 'r') as f:
            data = json.load(f)
            # Optional: Convert loaded datetime strings if needed for processing
            # _convert_datetime_strings_to_objects(data)
    except FileNotFoundError:
        return {"error": f"File not found: {json_file_path}"}
    except json.JSONDecodeError:
        return {"error": f"Invalid JSON format in file: {json_file_path}"}
    except Exception as e:
        return {"error": f"An error occurred reading the file: {e}"}

    if target_workflow_id not in data.get("workflows", {}):
        return {"error": f"Workflow ID '{target_workflow_id}' not found."}

    workflow_data = data["workflows"][target_workflow_id]
    analysis_results = {
        "workflow_id": target_workflow_id,
        "analysis_timestamp": datetime.datetime.utcnow().isoformat(),
        "samples": {}
    }

    for sample_id, sample_data in workflow_data.get("samples", {}).items():
        sample_analysis = {
            "sample_id": sample_id,
            "rerun_stats": {},
            "cleaned_successful_reruns": {} # Store cleaned paths here
        }

        for rerun_id, rerun_data in sample_data.get("reruns", {}).items():
            rerun_stats = {
                "rerun_id": rerun_id,
                "is_successful": False, # Overall goal achieved in *any* retry of *any* run
                "total_runs": 0,
                "successful_runs_count": 0, # Count of runs where *any* retry had actions_succeeded=True
                "successful_run_ids": [],
                "final_successful_run_retry_path": None # Path (rerun, run, retry) of the first retry achieving the goal
            }
            cleaned_runs_for_this_rerun = {} # Temp storage for cleaned runs of this rerun

            rerun_achieved_goal = False
            first_goal_achieving_path = None

            for run_id, run_data in rerun_data.get("runs", {}).items():
                rerun_stats["total_runs"] += 1
                run_had_successful_actions = False
                highest_successful_retry_id_int = -1
                successful_retry_node_for_cleaning = None # The node to clean for this run

                for run_retry_id, run_retry_data in run_data.get("run_retries", {}).items():
                    # Check if this retry achieved the overall goal for the rerun
                    if run_retry_data.get("overall_goal_success", False):
                        rerun_achieved_goal = True
                        # Store the path of the *first* retry that achieved the goal
                        if first_goal_achieving_path is None:
                             first_goal_achieving_path = (rerun_id, run_id, run_retry_id)

                    # Check if this retry had successful actions (for run success status and cleaning)
                    if run_retry_data.get("actions_succeeded", False):
                        run_had_successful_actions = True
                        try:
                            current_retry_id_int = int(run_retry_id)
                            # Keep track of the highest successful retry for cleaning purposes
                            if current_retry_id_int > highest_successful_retry_id_int:
                                highest_successful_retry_id_int = current_retry_id_int
                                successful_retry_node_for_cleaning = run_retry_data
                        except ValueError:
                            print(f"Warning: Non-integer run_retry_id '{run_retry_id}' found in {target_workflow_id}/{sample_id}/{rerun_id}/{run_id}. Skipping for cleaning.")
                            continue # Skip this retry for cleaning if ID isn't an int

                # --- Update Run Stats ---
                if run_had_successful_actions:
                    rerun_stats["successful_runs_count"] += 1
                    rerun_stats["successful_run_ids"].append(run_id)

                    # --- Build Cleaned Run (if run had successful actions) ---
                    if successful_retry_node_for_cleaning:
                        # 1. Deep copy the successful retry node
                        cleaned_run_retry_node = copy.deepcopy(successful_retry_node_for_cleaning)

                        # 2. Filter actions within the cleaned retry node
                        cleaned_actions = {}
                        if "actions" in cleaned_run_retry_node:
                            for action_id, action_data in cleaned_run_retry_node["actions"].items():
                                cleaned_candidates = {}
                                action_should_be_kept = False
                                if "candidates" in action_data:
                                    for candidate_id, candidate_data in action_data["candidates"].items():
                                        # Keep candidate ONLY if it was acted upon AND successful
                                        if candidate_data.get("to_act", False) and candidate_data.get("success", False):
                                            cleaned_candidates[candidate_id] = copy.deepcopy(candidate_data)
                                            action_should_be_kept = True # Mark action for keeping

                                # If at least one successful candidate was found, keep the action
                                if action_should_be_kept:
                                    cleaned_action_node = copy.deepcopy(action_data) # Keep action metadata
                                    cleaned_action_node["candidates"] = cleaned_candidates
                                    cleaned_actions[action_id] = cleaned_action_node

                        cleaned_run_retry_node["actions"] = cleaned_actions # Replace actions with cleaned ones

                        # 3. Create the cleaned run structure
                        cleaned_run_node = copy.deepcopy(run_data) # Keep run metadata
                        # Store ONLY the cleaned successful retry under its original ID (as string)
                        cleaned_run_node["run_retries"] = {str(highest_successful_retry_id_int): cleaned_run_retry_node}
                        cleaned_runs_for_this_rerun[run_id] = cleaned_run_node


            # --- Finalize Rerun Stats ---
            rerun_stats["is_successful"] = rerun_achieved_goal
            if rerun_achieved_goal:
                rerun_stats["final_successful_run_retry_path"] = first_goal_achieving_path

            if rerun_stats["total_runs"] > 0:
                rerun_stats["success_percentage"] = (rerun_stats["successful_runs_count"] / rerun_stats["total_runs"]) * 100.0
            else:
                rerun_stats["success_percentage"] = 0.0

            sample_analysis["rerun_stats"][rerun_id] = rerun_stats

            # --- Store Cleaned Rerun (if successful overall) ---
            if rerun_stats["is_successful"]:
                 # Create the cleaned rerun structure
                 cleaned_rerun_node = copy.deepcopy(rerun_data) # Keep rerun metadata
                 cleaned_rerun_node["runs"] = cleaned_runs_for_this_rerun # Add the cleaned runs
                 sample_analysis["cleaned_successful_reruns"][rerun_id] = cleaned_rerun_node


        analysis_results["samples"][sample_id] = sample_analysis

    # Optional: Convert datetime objects back to strings for JSON saving if needed elsewhere
    # analysis_results_serializable = _convert_datetime_objects_to_strings(analysis_results)
    # return analysis_results_serializable

    return analysis_results


def transform_to_guided_experience(analysis_results, original_workflow_data):
    """
    Transforms the cleaned successful rerun data from analysis_results
    into the specified guided experience JSON format.

    Args:
        analysis_results (dict): The output dictionary from analyze_workflow_logs.
        original_workflow_data (dict): The original data for the specific workflow
                                       from logger.json (needed for instructions).

    Returns:
        dict: A dictionary representing the guided experience JSON, or None if
              no successful path is found for any sample.
    """
    guided_experiences = {} # Store results per sample

    workflow_id = analysis_results.get("workflow_id")
    if not workflow_id:
        print("Error: Workflow ID missing in analysis results.")
        return None

    for sample_id, sample_analysis in analysis_results.get("samples", {}).items():
        # Find the first successful rerun for this sample
        successful_rerun_id = None
        cleaned_rerun_node = None
        if sample_analysis.get("cleaned_successful_reruns"):
            # Get the first key (rerun_id) from the cleaned reruns dictionary
            successful_rerun_id = next(iter(sample_analysis["cleaned_successful_reruns"]))
            cleaned_rerun_node = sample_analysis["cleaned_successful_reruns"][successful_rerun_id]

        if not cleaned_rerun_node:
            print(f"Warning: No cleaned successful rerun found for sample {sample_id}.")
            continue # Skip this sample if no successful path was cleaned

        # --- Start building the guided experience structure ---
        experience_id = f"{workflow_id}-guide-{sample_id}-{successful_rerun_id}" # Example ID generation
        guided_experience = {
            "id": experience_id,
            "description": original_workflow_data.get("workflow_instructions", f"Guided experience for {workflow_id}"),
            "topics": [], # As requested
            "few_shot_examples": [], # As requested
            "intent": workflow_id, # Using workflow_id as intent
            "steps": []
        }

        # --- Iterate through the cleaned runs and actions to build steps ---
        # Sort runs by run_id (assuming they are numeric strings)
        sorted_run_ids = sorted(cleaned_rerun_node.get("runs", {}).keys(), key=int)

        for run_id in sorted_run_ids:
            cleaned_run_node = cleaned_rerun_node["runs"][run_id]
            # There should only be one retry_id in the cleaned data
            if not cleaned_run_node.get("run_retries"):
                continue
            run_retry_id = next(iter(cleaned_run_node["run_retries"])) # Get the single retry ID
            cleaned_run_retry_node = cleaned_run_node["run_retries"][run_retry_id]

            # Sort actions by action_id (assuming they are numeric strings)
            sorted_action_ids = sorted(cleaned_run_retry_node.get("actions", {}).keys(), key=int)

            for action_id in sorted_action_ids:
                cleaned_action_node = cleaned_run_retry_node["actions"][action_id]
                # There should only be one candidate_id in the cleaned data
                if not cleaned_action_node.get("candidates"):
                    continue
                candidate_id = next(iter(cleaned_action_node["candidates"])) # Get the single candidate ID
                cleaned_candidate_node = cleaned_action_node["candidates"][candidate_id]

                # --- Extract data for the step ---
                step_id = f"{experience_id}-step-{action_id}" # Example step ID
                step_text = cleaned_action_node.get("description", "No action description provided.")
                step_friendly_text = cleaned_candidate_node.get("element_description", "No element description provided.")
                step_audio_id = step_id # Using step_id as audio_id

                # --- Extract mapping info (handle missing data) ---
                ui_selector = None
                identifier = None
                action_type = cleaned_candidate_node.get("action", "unknown") # Default action type

                metadata = cleaned_candidate_node.get("element_metadata")
                if isinstance(metadata, dict):
                    identifiers = metadata.get("identifiers")
                    if isinstance(identifiers, dict):
                        # Prefer CSS selector, fall back to XPath
                        ui_selector = identifiers.get("selector")
                        identifier = ui_selector # Use selector for both by default
                        if not ui_selector:
                            xpath_selector = identifiers.get("xpath")
                            if xpath_selector:
                                ui_selector = f"xpath={xpath_selector}" # Prefix XPath
                                identifier = ui_selector

                # --- Build highlighting and validation lists ---
                highlighting_list = []
                if ui_selector:
                    highlighting_list.append({"ui_selector": ui_selector})

                completeness_validation_list = []
                if identifier and action_type != "unknown":
                     # Only add validation if we have an identifier and a known action
                    completeness_validation_list.append({
                        "action_type": action_type,
                        "identifier": identifier
                    })

                # --- Create the step object ---
                step = {
                    "id": step_id,
                    "text": step_text,
                    "friendly_text": step_friendly_text,
                    "audio_id": step_audio_id, # Set as requested
                    "experience_mapping": {
                        "highlighting": highlighting_list,
                        "completeness_validation": completeness_validation_list
                    }
                }
                guided_experience["steps"].append(step)

        # Add the completed guided experience for this sample
        guided_experiences[sample_id] = guided_experience


    if not guided_experiences:
         return {"error": f"No successful reruns found to generate guided experience for workflow '{workflow_id}'."}

    # If you only want the guide for the *first* successful sample:
    # first_sample_id = next(iter(guided_experiences))
    # return guided_experiences[first_sample_id]

    # If you want guides for *all* successful samples:
    return guided_experiences




In [16]:
client = "intuit"

workflow_to_analyze = "add_cust_v98" # Use the workflow ID you want


json_file = Path(f"clients/{client}/logger.json") # Make sure this path is correct



# 1. Load original data to get workflow instructions
original_data = None
try:
    with open(json_file, 'r') as f:
        original_data = json.load(f)
except Exception as e:
    print(f"Error loading original data: {e}")
    exit()

if not original_data or workflow_to_analyze not in original_data.get("workflows", {}):
    print(f"Workflow '{workflow_to_analyze}' not found in original data.")
    exit()

original_workflow_data_for_analysis = original_data["workflows"][workflow_to_analyze]

# 2. Run the analysis function from the previous step
# Make sure the analyze_workflow_logs function is defined or imported
try:
    # Assuming analyze_workflow_logs is defined in this file or imported
    analysis = analyze_workflow_logs(json_file, workflow_to_analyze)
    if "error" in analysis:
            print(f"Analysis Error: {analysis['error']}")
            exit()

    analysis_file = Path(f"clients/{client}/{workflow_to_analyze}/analysis_{workflow_to_analyze}.json")
    
    try:
        # Ensure the directory exists and remains accessible
        analysis_file.parent.mkdir(parents=True, exist_ok=True,mode=0o755)
        with open(analysis_file, 'w') as f:
            json.dump(analysis, f, indent=2) # Save the analysis results
        print(f"\nAnalysis stats saved to: {analysis_file}")
    except Exception as e:
        print(f"\nError saving analysis stats file: {e}")



except NameError:
    print("Error: analyze_workflow_logs function not found. Make sure it's defined or imported.")
    exit()
except Exception as e:
    print(f"Error during analysis: {e}")
    exit()


# 3. Transform the analysis results
guided_experience_output = transform_to_guided_experience(
    analysis,
    original_workflow_data_for_analysis
)

# 4. Print or save the result
if guided_experience_output and "error" not in guided_experience_output:
    print("\n--- Generated Guided Experience JSON ---")
    # Print the guide for the first sample found (adjust if you want all)
    first_sample_key = next(iter(guided_experience_output))
    print(json.dumps(guided_experience_output[first_sample_key], indent=2))

    # Example: Save the first guide to a new file
    guide_file = Path(f"clients/{client}/{workflow_id}/guide_{workflow_to_analyze}_sample_{first_sample_key}.json")
    try:
        with open(guide_file, 'w') as f:
                json.dump(guided_experience_output[first_sample_key], f, indent=2)
        print(f"\nGuided experience saved to: {guide_file}")
    except Exception as e:
        print(f"\nError saving guided experience file: {e}")
elif guided_experience_output and "error" in guided_experience_output:
        print(f"\nTransformation Error: {guided_experience_output['error']}")
else:
    print("\nNo guided experience generated (likely no successful reruns found).")


# --- Mapping Summary & Skipped Fields ---

print("\n--- Field Mapping Summary ---")
print(f"- output.id: Generated based on workflow, sample, and successful rerun ID (e.g., '{workflow_to_analyze}-guide-0-0')")
print(f"- output.description: Taken from original_workflow_data['workflow_instructions']")
print(f"- output.topics: Set to [] (empty list)")
print(f"- output.few_shot_examples: Set to [] (empty list)")
print(f"- output.intent: Taken from analysis_results['workflow_id']")
print(f"- output.steps: Generated list based on successful actions in the cleaned path")
print(f"  - step.id: Generated based on experience_id and action_id (e.g., '{workflow_to_analyze}-guide-0-0-step-0')")
print(f"  - step.text: Taken from cleaned_action_node['description']")
print(f"  - step.friendly_text: Taken from cleaned_candidate_node['element_description']")
print(f"  - step.audio_id: Set to the same value as step.id")
print(f"  - step.experience_mapping.highlighting[0].ui_selector: Taken from cleaned_candidate_node['element_metadata']['identifiers']['selector'] (fallback to XPath if selector is missing)")
print(f"  - step.experience_mapping.completeness_validation[0].action_type: Taken from cleaned_candidate_node['action']")
print(f"  - step.experience_mapping.completeness_validation[0].identifier: Taken from cleaned_candidate_node['element_metadata']['identifiers']['selector'] (fallback to XPath if selector is missing)")

print("\n--- Skipped/Blank Fields ---")
print("- output.topics: Intentionally set to an empty list as requested.")
print("- output.few_shot_examples: Intentionally set to an empty list as requested.")
print("- step.audio_id: Set to step.id as a placeholder; no actual audio mapping performed.")
print("- Highlighting/Validation: These lists will be empty if the necessary 'element_metadata' or 'identifiers' were missing in the source JSON for a specific step.")
print("- Other metadata: Fields like 'created_at', 'open_log', specific run/retry details (beyond the successful path), error messages, advice, etc., from the original logger.json are *not* included in the final guided experience JSON structure.")


Analysis stats saved to: clients\intuit\add_cust_v98\analysis_add_cust_v98.json

--- Generated Guided Experience JSON ---
{
  "id": "add_cust_v98-guide-0-0",
  "description": "To add a new customer in QuickBooks Online, go to the Sales tab, then select Customers, and click New customer. Fill out the customer\u2019s details in the form and hit Save to finish.\n",
  "topics": [],
  "few_shot_examples": [],
  "intent": "add_cust_v98",
  "steps": [
    {
      "id": "add_cust_v98-guide-0-0-step-0",
      "text": "Click on the Sales tab in the left navigation menu",
      "friendly_text": "The 'Sales' menu item is located in the left navigation panel, approximately 1/3 of the way down the menu. It has a right-facing arrow indicating it can be expanded. It's positioned below 'Transactions' and above 'Expenses' in the menu hierarchy.",
      "audio_id": "add_cust_v98-guide-0-0-step-0",
      "experience_mapping": {
        "highlighting": [
          {
            "ui_selector": ".NavItem__N

C:\Users\dusan\AppData\Local\Temp\ipykernel_5456\1066280102.py:74: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "analysis_timestamp": datetime.datetime.utcnow().isoformat(),
